In [1]:
import numpy as np

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
corpus = ['king is a strong man', 'queen is a beautiful woman']

stop_words = stopwords

In [3]:
def remove_stop_words(corpus):
    stop_words = ['is', 'a']
    results = []
    for text in corpus:
        tmp = text.split(' ')
        for stop_word in stop_words:
            if stop_word in tmp:
                tmp.remove(stop_word)
        results.append(" ".join(tmp))
    return results

In [4]:
corpus = remove_stop_words(corpus)

words = []
for text in corpus:
    for word in text.split(' '):
        words.append(word)
        
words = set(words)
words

{'beautiful', 'king', 'man', 'queen', 'strong', 'woman'}

In [5]:
word2int = {}

for i, word in enumerate(words):
    word2int[word] = i

    sentences = []

for sentence in corpus:
    sentences.append(sentence.split())
    
WINDOW_SIZE = 2

data= []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        for neighbor in sentence[max(idx - WINDOW_SIZE, 0): min(idx + WINDOW_SIZE, len(sentence)) + 1]:
            if neighbor != word:
                data.append([word, neighbor])

In [6]:
import pandas as pd
for text in corpus:
    print(text)

king strong man
queen beautiful woman


In [7]:
df = pd.DataFrame(data, columns=['input', 'label'])
df.head(10)

,input,label
0,king,strong
1,king,man
2,strong,king
3,strong,man
4,man,king
5,man,strong
6,queen,beautiful
7,queen,woman
8,beautiful,queen
9,beautiful,woman


In [8]:
word2int

{'woman': 0, 'man': 1, 'king': 2, 'beautiful': 3, 'queen': 4, 'strong': 5}

In [11]:
import tensorflow as tf
import numpy as np
ONE_HOT_DIM = len(words)

def to_one_hot_encoding(data_point_index):
    one_hot_encoding = np.zeros(ONE_HOT_DIM)
    one_hot_encoding[data_point_index] = 1
    return one_hot_encoding

In [12]:
X = []
Y = []
for x, y in zip(df['input'], df['label']):
    X.append(to_one_hot_encoding(word2int[x]))
    Y.append(to_one_hot_encoding(word2int[y]))
    
X_train = np.asarray(X)
Y_train = np.asarray(Y)

x = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))
y_label = tf.placeholder(tf.float32, shape=(None, ONE_HOT_DIM))

In [24]:
EMBEDDING_DIM = 2

W1 = tf.Variable(tf.random_normal([ONE_HOT_DIM, EMBEDDING_DIM]))
b1 = tf.Variable(tf.random_normal([1]))
hidden_layer = tf.add(tf.matmul(x,W1),b1)

W2= tf.Variable(tf.random_normal([EMBEDDING_DIM, ONE_HOT_DIM]))
b2 = tf.Variable(tf.random_normal([1]))
prediction = tf.nn.softmax(tf.add(tf.matmul(hidden_layer, W2), b2))

loss = tf.reduce_mean(-tf.reduce_sum(y_label * tf.log(prediction), axis=[1]))

train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

In [25]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [26]:
iteration = 20000
for i in range(iteration):
    sess.run(train_op,feed_dict={x:X_train, y_label:Y_train})
    if i % 3000 ==0:
        print('iteration' + str(i)+ 'loss is:', sess.run(loss,feed_dict={x:X_train, y_label:Y_train}))

iteration0loss is: 4.3864007
iteration3000loss is: 0.90003204
iteration6000loss is: 0.87910366
iteration9000loss is: 0.8679581
iteration12000loss is: 0.8608642
iteration15000loss is: 0.8558824
iteration18000loss is: 0.85215706


In [27]:
vectors = sess.run(W1+b1)
print(vectors)

[[-0.48815975  0.36047533]
 [ 0.48260406 -0.35249928]
 [ 5.578891   -2.3634696 ]
 [-5.9818935   2.5801432 ]
 [-1.8739868  -1.8990886 ]
 [ 1.920494    1.9262718 ]]


In [28]:
w2v_df = pd.DataFrame(vectors, columns = ['x1', 'x2'])
w2v_df['word'] = words
w2v_df = w2v_df[['word', 'x1', 'x2']]
w2v_df

,word,x1,x2
0,woman,-0.488160,0.360475
1,man,0.482604,-0.352499
2,king,5.578891,-2.363470
3,beautiful,-5.981894,2.580143
4,queen,-1.873987,-1.899089
5,strong,1.920494,1.926272
